In [1]:
import sys
sys.path.append('..')

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import pandas as pd
tf.config.set_soft_device_placement(False)
tf.debugging.set_log_device_placement(True)
import numpy as np
from ampligraph.datasets import load_fb15k_237, load_yago3_10
from ampligraph.evaluation.protocol import create_mappings, to_idx

from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score
import time
print(tf.__version__)
assert(tf.__version__.startswith('2.4'))

from ampligraph.datasets import load_fb15k_237, load_fb13, load_fb15k, load_wn11, load_wn18, load_wn18rr, load_yago3_10
from ampligraph.latent_features import ScoringBasedEmbeddingModel

2.4.0


In [2]:
dataset = load_fb15k_237()

Jump to 
- [Partitioned](#Training/eval-with-partition)
- [Discovery](#Discovery)

# Train/eval without partition

In [3]:
optim = tf.optimizers.Adam(learning_rate=0.001)
# optim = 'adam'

# loss = nll
# loss = 'self_adversarial'

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='./logs')


from ampligraph.latent_features.loss_functions import SelfAdversarialLoss, NLLMulticlass
loss = SelfAdversarialLoss({'margin': 0.1, 'alpha': 5, 'reduction': 'sum'})
loss = NLLMulticlass({'reduction': 'mean'})
model = ScoringBasedEmbeddingModel(eta=1, 
                                     k=50,
                                     scoring_type='TransE')



model.compile(optimizer=optim, loss=loss)

from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('./chkpt1', monitor='loss', verbose=1, save_best_only=True, mode='min')

dataset = load_fb15k_237()

start = time.time()
model.fit(dataset['train'],
             batch_size=10000,
             epochs=10,
             validation_freq=25,
             validation_batch_size=100,
             validation_data = dataset['valid'],
         callbacks=[checkpoint, tensorboard_callback])

end = time.time()
print('Time taken:', end-start)

Epoch 1/10
29/29 [==============================] - 2s 57ms/step - loss: 6651.7305

Epoch 00001: loss improved from inf to 6651.73047, saving model to ./chkpt1
Epoch 2/10
29/29 [==============================] - 0s 6ms/step - loss: 6546.4531

Epoch 00002: loss improved from 6651.73047 to 6546.45312, saving model to ./chkpt1
Epoch 3/10
29/29 [==============================] - 0s 6ms/step - loss: 6421.3564

Epoch 00003: loss improved from 6546.45312 to 6421.35645, saving model to ./chkpt1
Epoch 4/10
29/29 [==============================] - 0s 6ms/step - loss: 6267.3594

Epoch 00004: loss improved from 6421.35645 to 6267.35938, saving model to ./chkpt1
Epoch 5/10
29/29 [==============================] - 0s 6ms/step - loss: 6093.7065

Epoch 00005: loss improved from 6267.35938 to 6093.70654, saving model to ./chkpt1
Epoch 6/10
29/29 [==============================] - 0s 6ms/step - loss: 5913.9697

Epoch 00006: loss improved from 6093.70654 to 5913.96973, saving model to ./chkpt1
Epoch 7/10

### Visualize the embeddings

In [4]:
from ampligraph.utils import create_tensorboard_visualizations

In [5]:
create_tensorboard_visualizations(model, 
                                  entities_subset=['/m/027rn', '/m/06cx9', '/m/017dcd', '/m/06v8s0', '/m/07s9rl0'], 
                                  labels=['ent1', 'ent2', 'ent3', 'ent4', 'ent5'],
                                  loc = './small_embeddings_vis')

In [6]:
create_tensorboard_visualizations(model, 
                                  entities_subset='all',
                                  loc = './full_embeddings_vis')

In [7]:
# the embeddings can be visualised using the following command:
# tensorboard --logdir='./full_embeddings_vis' --port=8891 
# open the browser and go to the following URL: http://127.0.0.1:8891/#projector

### Evaluate

In [6]:
# evaluate using filters
ranks = model.evaluate(np.array([['/m/01cr28', '/location/country/form_of_government', '/m/02lkcc'],
                     ['/m/07tw_b', '/location/country/form_of_government', '/m/02lkcc'],
                     ['/m/073tm9', '/location/country/form_of_government', '/m/02lkcc']]), 
                       use_filter={'train': dataset['train']}, 
                       corrupt_side='s,o', 
                       verbose=True)

2/2 [==============================] - 2s 936ms/step


In [ ]:
ranks

In [ ]:
# evaluate using filters (filters are file names instead of numpy arrays)
# corruptions generated using entities subset
start = time.time()
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
                       entities_subset=['/m/08966', '/m/05lf_', '/m/0f8l9c', '/m/04ghz4m'],
                      
                      use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
mrr_score(ranks) # will give very high mrr

In [5]:
# Full evaluation (default protocol) using filters
start = time.time()
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
                      use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
mrr_score(ranks)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
206/206 [==============================] - 69s 334ms/step


0.28979677376532464

In [6]:
# same as above but just for sanity checking if entities_subset works or not
start = time.time()
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
                       entities_subset=list(model.data_indexer.backend.entities_dict.values()),
                      
                      use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mrr_score(ranks)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
206/206 [==============================] - 70s 342ms/step
Time taken: 70.50386095046997


0.28979677376532464

In [9]:
np.random.seed(0)
pred = model.predict('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100)
print(pred.shape)
print(np.sort(pred))


28 triples containing invalid keys skipped!
(20438,)
[-2.6117423 -2.5582662 -2.434757  ... 30.218779  31.553257  41.351562 ]


### Model calibration

In [10]:
# calibrate on the test set
model.calibrate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                batch_size=10000, positive_base_rate=0.9, epochs=100)


28 triples containing invalid keys skipped!


In [11]:
# check if the sorted probability indices match the sorted regular scores 
# It should be same as calibration doesnt change ranking, it just calibrates the range of scores
out = model.predict_proba('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', batch_size=10000)
print(np.sort(out))
print(np.argsort(out))
pred_out = model.predict('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt')
print(np.sort(pred_out))
print(np.argsort(pred_out))



28 triples containing invalid keys skipped!
[0.4154276  0.41815606 0.42447674 ... 0.9985715  0.9989201  0.9998617 ]
[ 4066   611 18634 ...   990 10437 14612]

28 triples containing invalid keys skipped!
[-2.6117423 -2.5582662 -2.434757  ... 30.218779  31.553257  41.351562 ]
[ 4066   611 18634 ...   990 10437 14612]


In [12]:
# calibration should not affect the regular evaluation
start = time.time()
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
         use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
206/206 [==============================] - 66s 321ms/step
Time taken: 66.17234683036804


(298.0980771112633,
 0.22008817445328427,
 0.13516488893238085,
 0.39289558665231433,
 20438)

In [13]:
model.save_weights('./calibrated_model')

## Load Checkpoint and evaluate

In [14]:
# Loaded the model should return exact same results as earlier
start = time.time()
loaded_model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     scoring_type='ComplEx')
loaded_model.load_weights('./calibrated_model')
ranks = loaded_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
         use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
206/206 [==============================] - 66s 321ms/step
Time taken: 66.22395181655884


(298.0980771112633,
 0.22008817445328427,
 0.13516488893238085,
 0.39289558665231433,
 20438)

In [15]:
pred = loaded_model.predict('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100)
print(pred.shape)
print(pred)


28 triples containing invalid keys skipped!
(20438,)
[14.611031   9.369467   7.550662  ...  5.5516796  0.9343726  5.4705048]


In [16]:
# sorted calibration scores order and regular predict scores order must match
out = loaded_model.predict_proba('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', batch_size=10000)
np.argsort(out), np.argsort(loaded_model.predict('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt')), np.sort(out)


28 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!


(array([ 4066,   611, 18634, ...,   990, 10437, 14612]),
 array([ 4066,   611, 18634, ...,   990, 10437, 14612]),
 array([0.4154276 , 0.41815606, 0.42447674, ..., 0.9985715 , 0.9989201 ,
        0.9998617 ], dtype=float32))

# Training/eval with partition 

## Training with RandomEdges partitioner

In [17]:
from ampligraph.datasets import DummyBackend, SQLiteAdapter
from ampligraph.datasets import GraphDataLoader
from ampligraph.datasets.graph_partitioner import PARTITION_ALGO_REGISTRY
dataset_loader = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt', 
                                 backend=SQLiteAdapter,
                                    batch_size=1000, 
                                    dataset_type='train', 
                                     use_filter=False,
                                    use_indexer=True)

In [18]:
# Choose the partitioner 
partitioner = PARTITION_ALGO_REGISTRY.get('RandomEdges')(dataset_loader, k=3)


_split: memory before: 896.0Bytes, after: 12.916MB, consumed: 12.915MB; exec time: 31.859s


In [19]:

optim = tf.optimizers.Adam(learning_rate=0.001, amsgrad=True)

partitioned_model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     scoring_type='TransE')
partitioned_model.compile(optimizer=optim, loss='multiclass_nll')

In [20]:

start = time.time()
partitioned_model.fit(partitioner,
                     batch_size=1000, use_partitioning=True,             
                     epochs=10)
print((time.time()-start))

Epoch 1/10
274/274 [==============================] - 19s 68ms/step - loss: 1587.2365
Epoch 2/10
274/274 [==============================] - 18s 67ms/step - loss: 1413.6970
Epoch 3/10
274/274 [==============================] - 19s 68ms/step - loss: 1251.7213
Epoch 4/10
274/274 [==============================] - 18s 67ms/step - loss: 1116.2098
Epoch 5/10
274/274 [==============================] - 19s 69ms/step - loss: 1007.5591
Epoch 6/10
274/274 [==============================] - 18s 66ms/step - loss: 920.2757
Epoch 7/10
274/274 [==============================] - 18s 66ms/step - loss: 849.2607
Epoch 8/10
274/274 [==============================] - 18s 66ms/step - loss: 790.6967
Epoch 9/10
274/274 [==============================] - 18s 66ms/step - loss: 741.4282
Epoch 10/10
274/274 [==============================] - 18s 67ms/step - loss: 699.5930
196.18159699440002


In [21]:

dataset_loader_test = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                        backend=SQLiteAdapter,
                                        batch_size=400, 
                                        dataset_type='test', 
                                        use_indexer=partitioned_model.data_handler.get_mapper())


28 triples containing invalid keys skipped!


In [22]:

start = time.time()
ranks = partitioned_model.evaluate(dataset_loader_test, 
                                   batch_size=400)
end = time.time()
print('Time taken:', end-start)

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)

53/53 [==============================] - 128s 2s/step
Time taken: 128.40124201774597


(464.99716214893823, 0.09894986543567783, 0.0, 0.2728251296604364, 20438)

## Save and Load

In [23]:
partitioned_model.save_weights('./best_model')

In [24]:
loaded_part_model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     scoring_type='TransE')

loaded_part_model.load_weights('./best_model')

In [25]:

dataset_loader_test = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                        backend=DummyBackend,
                                        batch_size=400, 
                                        dataset_type='test', 
                                        use_indexer=loaded_part_model.data_indexer)


28 triples containing invalid keys skipped!


In [26]:


start = time.time()
ranks = loaded_part_model.evaluate(dataset_loader_test, 
                                   batch_size=400)
end = time.time()
print('Time taken:', end-start)

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)

53/53 [==============================] - 128s 2s/step
Time taken: 128.0037112236023


(464.99716214893823, 0.09894986543567783, 0.0, 0.2728251296604364, 20438)

# Training/eval with partition (default Partitioning Approach)

In [47]:
from ampligraph.datasets import DummyBackend, SQLiteAdapter
from ampligraph.datasets import GraphDataLoader
from ampligraph.datasets.graph_partitioner import PARTITION_ALGO_REGISTRY


In [48]:
optim = tf.optimizers.Adam(learning_rate=0.0001, amsgrad=True)

partitioned_model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     scoring_type='TransE')
partitioned_model.compile(optimizer=optim, loss='multiclass_nll')




In [46]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='./logs_split')
start = time.time()
partitioned_model.fit('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                     batch_size=10000, use_partitioning=True,
                     epochs=100, callbacks=[tensorboard_callback])
print((time.time()-start))

_split: memory before: 316.36MB, after: 296.09MB, consumed: -2.1259e+07Bytes; exec time: 87.255s
Epoch 1/100
31/31 [==============================] - 8s 266ms/step - loss: 16094.2588
Epoch 2/100
31/31 [==============================] - 8s 257ms/step - loss: 16101.5303
Epoch 3/100
31/31 [==============================] - 8s 266ms/step - loss: 16103.0107
Epoch 4/100
31/31 [==============================] - 8s 265ms/step - loss: 16099.6152
Epoch 5/100
31/31 [==============================] - 8s 260ms/step - loss: 16092.4268
Epoch 6/100
31/31 [==============================] - 8s 262ms/step - loss: 16082.4639
Epoch 7/100
31/31 [==============================] - 8s 266ms/step - loss: 16070.2266
Epoch 8/100
31/31 [==============================] - 8s 257ms/step - loss: 16056.2168
Epoch 9/100
31/31 [==============================] - 8s 263ms/step - loss: 16040.7246
Epoch 10/100
31/31 [==============================] - 8s 269ms/step - loss: 16023.9854
Epoch 11/100
31/31 [======================

KeyboardInterrupt: 

In [44]:
len(set(dataset['train'][:, 0]).union(set(dataset['train'][:, 2])))

14505

In [45]:


start = time.time()
ranks = partitioned_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                   batch_size=400)
end = time.time()
print('Time taken:', end-start)

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)


28 triples containing invalid keys skipped!
      6/Unknown - 14s 2s/step

KeyboardInterrupt: 

## Filtered evaluation

In [31]:
start = time.time()
ranks = partitioned_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=400,
                       corrupt_side='s,o',
                        use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                              'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                              'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
      6/Unknown - 35s 6s/step

KeyboardInterrupt: 

# Discovery

### Discover Facts

In [ ]:
from ampligraph.discovery import discover_facts

model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300,
                                     scoring_type='ComplEx')



model.compile(optimizer='adam', loss='multiclass_nll')

start = time.time()
model.fit(dataset['train'],
             batch_size=10000,
             epochs=10,
             validation_freq=50,
             validation_batch_size=100,
             validation_data = dataset['valid'])

end = time.time()
print('Time taken:', end-start)

discover_facts(dataset['train'][:100], 
               model, 
               top_n=100, 
               strategy='random_uniform', 
               max_candidates=100, 
               target_rel='/location/country/form_of_government', 
               seed=0)

### Find Clusters

In [ ]:
import requests
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns


from ampligraph.datasets import load_from_csv
from ampligraph.discovery import find_clusters

# International football matches triples
# See tutorial here to understand how the triples are created from a tabular dataset:
url = 'https://ampligraph.s3-eu-west-1.amazonaws.com/datasets/football.csv'
open('football.csv', 'wb').write(requests.get(url).content)
X = load_from_csv('.', 'football.csv', sep=',')[:, 1:]

model = ScoringBasedEmbeddingModel(eta=5, 
                                 k=300,
                                 scoring_type='ComplEx')



model.compile(optimizer='adam', loss='multiclass_nll')

model.fit(X,
          batch_size=10000,
          epochs=10)

df = pd.DataFrame(X, columns=["s", "p", "o"])
teams = np.unique(np.concatenate((df.s[df.s.str.startswith("Team")],
                               df.o[df.o.str.startswith("Team")])))
team_embeddings = model.get_embeddings(teams, embedding_type='e')

embeddings_2d = PCA(n_components=2).fit_transform(np.array([i for i in team_embeddings]))

# Find clusters of embeddings using KMeans

kmeans = KMeans(n_clusters=6, n_init=100, max_iter=500)
clusters = find_clusters(teams, model, kmeans, mode='e')
print(np.unique(clusters, return_counts=True))

### Find Duplicates

In [ ]:
import pandas as pd
import numpy as np
import re

# The IMDB dataset used here is part of the Movies5 dataset found on:
# The Magellan Data Repository (https://sites.google.com/site/anhaidgroup/projects/data)
import requests
url = 'http://pages.cs.wisc.edu/~anhai/data/784_data/movies5.tar.gz'
open('movies5.tar.gz', 'wb').write(requests.get(url).content)
import tarfile
tar = tarfile.open('movies5.tar.gz', "r:gz")
tar.extractall()
tar.close()

# Reading tabular dataset of IMDB movies and filling the missing values
imdb = pd.read_csv("movies5/csv_files/imdb.csv")
imdb["directors"] = imdb["directors"].fillna("UnknownDirector")
imdb["actors"] = imdb["actors"].fillna("UnknownActor")
imdb["genre"] = imdb["genre"].fillna("UnknownGenre")
imdb["duration"] = imdb["duration"].fillna("0")

# Creating knowledge graph triples from tabular dataset
imdb_triples = []

for _, row in imdb.iterrows():
    movie_id = "ID" + str(row["id"])
    directors = row["directors"].split(",")
    actors = row["actors"].split(",")
    genres = row["genre"].split(",")
    duration = "Duration" + str(int(re.sub("\D", "", row["duration"])) // 30)

    directors_triples = [(movie_id, "hasDirector", d) for d in directors]
    actors_triples = [(movie_id, "hasActor", a) for a in actors]
    genres_triples = [(movie_id, "hasGenre", g) for g in genres]
    duration_triple = (movie_id, "hasDuration", duration)

    imdb_triples.extend(directors_triples)
    imdb_triples.extend(actors_triples)
    imdb_triples.extend(genres_triples)
    imdb_triples.append(duration_triple)

# Training knowledge graph embedding with ComplEx model
model = ScoringBasedEmbeddingModel(eta=5, 
                             k=300,
                             scoring_type='ComplEx')



model.compile(optimizer='adam', loss='multiclass_nll')
imdb_triples = np.array(imdb_triples)
model.fit(imdb_triples,
      batch_size=10000,
      epochs=20)

# Finding duplicates movies (entities)
from ampligraph.discovery import find_duplicates

entities = np.unique(imdb_triples[:, 0])
dups, _ = find_duplicates(entities, model, mode='e', tolerance=0.45)
id_list = []
for data in dups:
    for i in data:
        id_list.append(int(i[2:]))
print(imdb.iloc[id_list[:6]][['movie_name', 'year']])

### Query TopN

In [ ]:
import requests
from ampligraph.datasets import load_from_csv
from ampligraph.discovery import discover_facts
from ampligraph.discovery import query_topn

# Game of Thrones relations dataset
url = 'https://ampligraph.s3-eu-west-1.amazonaws.com/datasets/GoT.csv'
open('GoT.csv', 'wb').write(requests.get(url).content)
X = load_from_csv('.', 'GoT.csv', sep=',')

model = ScoringBasedEmbeddingModel(eta=5, 
                             k=150,
                             scoring_type='TransE')



model.compile(optimizer='adagrad', loss='pairwise')
model.fit(X,
      batch_size=100,
      epochs=20)
query_topn(model, top_n=5,
        head='Eddard Stark', relation='ALLIED_WITH', tail=None,
        ents_to_consider=None, rels_to_consider=None)
